In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests
import spotipy
import sys
import pprint
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
with open('grammyData.json') as json_data:
    d = json.load(json_data)
    print(d)
sep = '::::' # string that doesn't appear in column names

df_grammy_year = json_normalize(d, sep=sep)
df_grammy_year.columns = df_grammy_year.columns.str.split(sep).str[-1]
df_grammy_year.to_csv("Grammy_Year.csv",index = False)

[{'category': 'Record Of The Year', 'annualGrammy': 1, 'awardType': 'Individual', 'name': 'Nel Blu Dipinto Di Blu (Volare)', 'awardFor': 'Nel Blu Dipinto Di Blu (Volare)'}, {'category': 'Album Of The Year', 'annualGrammy': 1, 'awardType': 'Work', 'name': 'Henry Mancini', 'awardFor': 'The Music From Peter Gunn'}, {'category': 'Song Of The Year', 'annualGrammy': 1, 'awardType': 'Individual', 'name': 'Nel Blu Dipinto Di Blu (Volare)', 'awardFor': 'Nel Blu Dipinto Di Blu (Volare)'}, {'category': 'Best Vocal Performance, Female', 'annualGrammy': 1, 'awardType': 'Work', 'name': 'Ella Fitzgerald', 'awardFor': 'Ella Fitzgerald Sings The Irving Berlin Song Book'}, {'category': 'Best Vocal Performance, Male', 'annualGrammy': 1, 'awardType': 'Individual', 'name': 'Catch A Falling Star', 'awardFor': 'Catch A Falling Star'}, {'category': 'Best Performance By An Orchestra', 'annualGrammy': 1, 'awardType': 'Individual', 'name': "Billy May's Big Fat Brass", 'awardFor': "Billy May's Big Fat Brass"}, {'

In [3]:
df = pd.DataFrame.from_dict(json_normalize(d), orient='columns')
df_record_of_the_year = df.loc[df["category"] == "Record Of The Year", :]
df_record_of_the_year
winner_list = df_record_of_the_year["awardFor"].tolist()
#replace 'Stay With Me (Darkchild Version)' to 'Stay With Me'
winner_list[51]="Stay With Me"
winner_list

['Nel Blu Dipinto Di Blu (Volare)',
 'Mack The Knife',
 'Moon River',
 'I Left My Heart In San Francisco',
 'Days Of Wine And Roses',
 'The Girl From Ipanema',
 'A Taste Of Honey',
 'Strangers In The Night',
 'Up, Up And Away',
 'Mrs. Robinson',
 'Aquarius/Let The Sunshine In (The Flesh Failures)',
 'Bridge Over Troubled Water',
 "It's Too Late",
 'The First Time Ever I Saw Your Face',
 'Killing Me Softly With His Song',
 'I Honestly Love You',
 'Love Will Keep Us Together',
 'This Masquerade',
 'Hotel California',
 'Just The Way You Are',
 'What A Fool Believes',
 'Bette Davis Eyes',
 'Rosanna',
 'Beat It',
 "What's Love Got To Do With It",
 'We Are The World',
 'Higher Love',
 'Graceland',
 "Don't Worry Be Happy",
 'Wind Beneath My Wings',
 'Unforgettable',
 'Tears In Heaven',
 'I Will Always Love You',
 'All I Wanna Do',
 'Kiss From A Rose',
 'Change The World',
 'Sunny Came Home',
 'My Heart Will Go On (Love Theme From Titanic)',
 'Smooth',
 'Walk On',
 "Don't Know Why",
 'Clocks',

In [4]:
annualGrammy_list = df_record_of_the_year["annualGrammy"].tolist()
annualGrammy_list

[1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 3,
 13,
 23,
 33,
 43,
 53]

In [5]:
#spotify API 
client_credentials_manager = SpotifyClientCredentials("9cc473d2e26f41849b4921a2db73af8a","56f41bc448d64af187db063121d6b82f")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Genius API
base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer '+ '7liNvoUiTMLf18YUqr6UvkCUs-aNzu_N-Go-1AyN7IJDotFSAP6gvIFuxvonmxGo'}
search_url = base_url + '/search'

winner_features=[]

for winner in winner_list:
    try:
        result = sp.search(winner,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":winner,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "features":features,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "lyric_total_word_count":lyrice_total_count,
            "lyric_unique_word_count":lyrice_unique_count,
            "win/loss":1
        }
    
        winner_features.append(song_features)
    except:
        pass    
    
print(winner_features)
    



retrying ...1secs
retrying ...1secs
retrying ...1secs
[{'song_name': 'Nel Blu Dipinto Di Blu (Volare)', 'track': 'Nel blu dipinto di blu', 'artist': 'Domenico Modugno', 'song_id': '006Ndmw2hHxvnLbJsBFnPx', 'popularity': 34, 'duration_ms': 216373, 'release_date': '2008-02-28', 'release_year': 2008, 'features': {'danceability': 0.518, 'energy': 0.06, 'key': 10, 'loudness': -14.887, 'mode': 1, 'speechiness': 0.0441, 'acousticness': 0.987, 'instrumentalness': 7.87e-06, 'liveness': 0.161, 'valence': 0.336, 'tempo': 127.87, 'type': 'audio_features', 'id': '006Ndmw2hHxvnLbJsBFnPx', 'uri': 'spotify:track:006Ndmw2hHxvnLbJsBFnPx', 'track_href': 'https://api.spotify.com/v1/tracks/006Ndmw2hHxvnLbJsBFnPx', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/006Ndmw2hHxvnLbJsBFnPx', 'duration_ms': 216373, 'time_signature': 4}, 'artist_followers': 58482, 'artist_popularity': 50, 'lyric_total_word_count': 244, 'lyric_unique_word_count': 108, 'win/loss': 1}, {'song_name': 'Mack The Knife', 'trac

In [14]:
sep = '::::' # string that doesn't appear in column names

df_winner_features = json_normalize(winner_features, sep=sep)
df_winner_features.columns = df_winner_features.columns.str.split(sep).str[-1]
df_winner_features.to_csv("Record of Year Winner Features.csv",index = False)

In [15]:
df_winner_features.head(10)

,artist,artist_followers,artist_popularity,duration_ms,acousticness,analysis_url,danceability,duration_ms,energy,id,...,valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,Domenico Modugno,58482,50,216373,0.9870,https://api.spotify.com/v1/audio-analysis/006N...,0.518,216373,0.060,006Ndmw2hHxvnLbJsBFnPx,...,0.3360,244,108,34,2008-02-28,2008,006Ndmw2hHxvnLbJsBFnPx,Nel Blu Dipinto Di Blu (Volare),Nel blu dipinto di blu,1
1,Bobby Darin,170045,62,184333,0.7600,https://api.spotify.com/v1/audio-analysis/0YTg...,0.549,184333,0.529,0YTgZJfVLmJ0b8XAMZKRKg,...,0.4640,279,136,56,2004-11-23,2004,0YTgZJfVLmJ0b8XAMZKRKg,Mack The Knife,Mack The Knife,1
2,Frank Ocean,3864517,83,188323,0.8770,https://api.spotify.com/v1/audio-analysis/41cp...,0.240,188324,0.116,41cpvQ2GyGb2BRdIRSsTqK,...,0.0937,178,72,67,2018-02-14,2018,41cpvQ2GyGb2BRdIRSsTqK,Moon River,Moon River,1
3,Tony Bennett,369991,71,170960,0.9550,https://api.spotify.com/v1/audio-analysis/1bYI...,0.313,170960,0.078,1bYItjcPRtxZs6Ir71O5RB,...,0.1250,110,78,54,1962-06-18,1962,1bYItjcPRtxZs6Ir71O5RB,I Left My Heart In San Francisco,(I Left My Heart) In San Francisco,1
4,Frank Sinatra,2544316,82,225426,0.8670,https://api.spotify.com/v1/audio-analysis/2LnT...,0.282,225427,0.113,2LnTydfDh8VOvJH7cSLqjS,...,0.2250,130,46,51,1964,1964,2LnTydfDh8VOvJH7cSLqjS,Days Of Wine And Roses,Three Coins In The Fountain,1
5,Amy Winehouse,5145551,79,166960,0.2500,https://api.spotify.com/v1/audio-analysis/7F5r...,0.696,166960,0.616,7F5rIyOI88zk36AiBXC8xq,...,0.6600,220,77,61,2011-01-01,2011,7F5rIyOI88zk36AiBXC8xq,The Girl From Ipanema,The Girl From Ipanema,1
6,Paul Desmond,47563,55,271000,0.9180,https://api.spotify.com/v1/audio-analysis/6PdC...,0.492,271000,0.151,6PdC87pNlAOBZFRVhk2c3Q,...,0.2140,5,1,49,1997-08-26,1997,6PdC87pNlAOBZFRVhk2c3Q,A Taste Of Honey,A Taste of Honey - Live,1
7,Frank Sinatra,2544316,82,157866,0.6380,https://api.spotify.com/v1/audio-analysis/74VR...,0.229,157867,0.494,74VR3AkGPhbYXnxcOYa16x,...,0.5260,176,82,65,1966-05,1966,74VR3AkGPhbYXnxcOYa16x,Strangers In The Night,Strangers In The Night,1
8,Kid Cudi,2741042,78,227346,0.0727,https://api.spotify.com/v1/audio-analysis/1RUT...,0.660,227347,0.875,1RUTIdTnFs8lHSc0Zr4UJB,...,0.6110,645,199,56,2009-01-01,2009,1RUTIdTnFs8lHSc0Zr4UJB,"Up, Up And Away",Up Up & Away,1
9,Simon & Garfunkel,1534809,74,244026,0.7130,https://api.spotify.com/v1/audio-analysis/0iOZ...,0.606,244027,0.457,0iOZM63lendWRTTeKhZBSC,...,0.8130,361,128,73,1968-04-03,1968,0iOZM63lendWRTTeKhZBSC,Mrs. Robinson,Mrs. Robinson,1


In [8]:
#Read Billboard 2010 - 2017 Top Song Date(Exclude Grammy Record Winner)
df_billboard_list = pd.read_csv("songs_2000_2017_Non_Grammy.csv")
df_billboard_list.head()

,date,title,artist,peakPos,lastPos,weeks,rank,change
0,10/28/17,Almost Like Praying,Lin-Manuel Miranda Featuring Artists For Puert...,20,0,1,20,Hot Shot Debut
1,10/28/17,Loyalty.,Kendrick Lamar Featuring Rihanna,14,44,26,48,-4
2,10/21/17,Redbone,Childish Gambino,12,43,44,50,-7
3,10/14/17,I'm The One,"DJ Khaled Featuring Justin Bieber, Quavo, Chan...",1,39,22,42,-3
4,10/14/17,Stay,Zedd & Alessia Cara,7,42,31,47,-5


In [9]:
#get only the list of title column
billboard_list = df_billboard_list["title"].tolist()
billboard_list
len(billboard_list)

1412

In [10]:
#get song features by Spotify
billboard_features=[]


for song in billboard_list:
    try:
        result = sp.search(song,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":song,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "features":features,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "lyric_total_word_count":lyrice_total_count,
            "lyric_unique_word_count":lyrice_unique_count,
            "win/loss":0
        }
    
        billboard_features.append(song_features)
    except:
            pass    
    
print(billboard_features)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
[{'song_name': 'Almost Like Praying', 'track': 'Almost Like Praying (feat. Artists for Puerto Rico)', 'artist': 'Lin-Manuel Miranda', 'song_id': '3dskm5fAe8OkBtM9E0VVWx', 'popularity': 51, 'duration_ms': 200457, 'release_date': '2017-10-06', 'release_year': 2017, 'features': {'danceability': 0.471, 'energy': 0.791, 'key': 5, 'loudness': -3.373, 'mode': 0, 'speechiness': 0.238, 'acousticness': 0.0198, 'instrumentalness': 0, 'liveness': 0.232, 'valence': 0.507, 'tempo': 188.36, 'type': 'audio_features', 'id': '3dskm5fAe8OkBtM9E0VVWx', 'uri': 'spotify:track:3dskm5fAe8OkBtM9E0VVWx', 'track_href': 'https://api.spotify.com/v1/tracks/3dskm5fAe8OkBtM9E0VVWx'

In [11]:
sep = '::::' # string that doesn't appear in column names

df_billboard_features = json_normalize(billboard_features, sep=sep)
df_billboard_features.columns = df_billboard_features.columns.str.split(sep).str[-1]
df_billboard_features.head(20)

,artist,artist_followers,artist_popularity,duration_ms,acousticness,analysis_url,danceability,duration_ms,energy,id,...,valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,Lin-Manuel Miranda,155413,77,200457,0.01980,https://api.spotify.com/v1/audio-analysis/3dsk...,0.471,200457.0,0.791,3dskm5fAe8OkBtM9E0VVWx,...,0.507,350,246,51,2017-10-06,2017,3dskm5fAe8OkBtM9E0VVWx,Almost Like Praying,Almost Like Praying (feat. Artists for Puerto ...,0
1,Trippie Redd,2188584,91,142648,0.00212,https://api.spotify.com/v1/audio-analysis/284f...,0.808,142649.0,0.701,284fVGdbsPtigjjI0jmKXo,...,0.154,409,154,76,2018-11-09,2018,284fVGdbsPtigjjI0jmKXo,Loyalty.,Loyalty Before Royalty,0
2,Childish Gambino,4086015,82,326933,0.17200,https://api.spotify.com/v1/audio-analysis/3kxf...,0.745,326933.0,0.356,3kxfsdsCpFgN412fpnW85Y,...,0.607,356,118,81,2016-12-02,2016,3kxfsdsCpFgN412fpnW85Y,Redbone,Redbone,0
3,Sam Smith,7340936,90,239316,0.52900,https://api.spotify.com/v1/audio-analysis/7795...,0.677,239317.0,0.485,7795WJLVKJoAyVoOtCWqXN,...,0.493,306,101,82,2014-01-01,2014,7795WJLVKJoAyVoOtCWqXN,I'm The One,I'm Not The Only One,0
4,YG,1416297,83,237240,0.00552,https://api.spotify.com/v1/audio-analysis/0ZNr...,0.745,237240.0,0.346,0ZNrc4kNeQYD9koZ3KvCsy,...,0.112,551,364,85,2018-08-03,2018,0ZNrc4kNeQYD9koZ3KvCsy,Stay,"BIG BANK (feat. 2 Chainz, Big Sean, Nicki Minaj)",0
5,Future,7374663,92,204600,0.01020,https://api.spotify.com/v1/audio-analysis/0Vgk...,0.833,204600.0,0.434,0VgkVdmE4gld66l8iyGjgx,...,0.281,653,228,81,2017-06-30,2017,0VgkVdmE4gld66l8iyGjgx,Mask Off,Mask Off,0
6,Kygo,3961834,88,220780,0.11900,https://api.spotify.com/v1/audio-analysis/12GE...,0.640,220781.0,0.533,12GEpg2XOPyqk03JZEZnJs,...,0.515,552,116,70,2017-02-17,2017,12GEpg2XOPyqk03JZEZnJs,It Ain't Me,It Ain’t Me (with Selena Gomez),0
7,Ed Sheeran,36259880,93,261153,0.02320,https://api.spotify.com/v1/audio-analysis/6PCU...,0.461,261154.0,0.834,6PCUP3dWmTjcTtXY02oFdT,...,0.471,392,153,81,2017-03-03,2017,6PCUP3dWmTjcTtXY02oFdT,Castle On The Hill,Castle on the Hill,0
8,A Boogie Wit da Hoodie,2120932,81,209269,0.50100,https://api.spotify.com/v1/audio-analysis/1f5c...,0.839,209269.0,0.810,1f5cbQtDrykjarZVrShaDI,...,0.814,718,460,79,2017-09-29,2017,1f5cbQtDrykjarZVrShaDI,Drowning,Drowning (feat. Kodak Black),0
9,YFN Lucci,955786,70,197170,0.12800,https://api.spotify.com/v1/audio-analysis/4Pzo...,0.758,197171.0,0.646,4PzovBqgnSHKd8opsP7IVM,...,0.589,590,243,70,2017-03-03,2017,4PzovBqgnSHKd8opsP7IVM,Everyday We Lit,Everyday We Lit (feat. PnB Rock),0


In [12]:
df_billboard_features.to_csv("Billboard_Non_Grammy_Song_Features_2000.csv",index = False)

In [16]:
#Combin grammy and billboard song data
full_song_list = pd.concat([df_billboard_features, df_winner_features], ignore_index=True)
full_song_list.head()

,artist,artist_followers,artist_popularity,duration_ms,acousticness,analysis_url,danceability,duration_ms,energy,id,...,valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,Lin-Manuel Miranda,155413,77,200457,0.01980,https://api.spotify.com/v1/audio-analysis/3dsk...,0.471,200457.0,0.791,3dskm5fAe8OkBtM9E0VVWx,...,0.507,350,246,51,2017-10-06,2017,3dskm5fAe8OkBtM9E0VVWx,Almost Like Praying,Almost Like Praying (feat. Artists for Puerto ...,0
1,Trippie Redd,2188584,91,142648,0.00212,https://api.spotify.com/v1/audio-analysis/284f...,0.808,142649.0,0.701,284fVGdbsPtigjjI0jmKXo,...,0.154,409,154,76,2018-11-09,2018,284fVGdbsPtigjjI0jmKXo,Loyalty.,Loyalty Before Royalty,0
2,Childish Gambino,4086015,82,326933,0.17200,https://api.spotify.com/v1/audio-analysis/3kxf...,0.745,326933.0,0.356,3kxfsdsCpFgN412fpnW85Y,...,0.607,356,118,81,2016-12-02,2016,3kxfsdsCpFgN412fpnW85Y,Redbone,Redbone,0
3,Sam Smith,7340936,90,239316,0.52900,https://api.spotify.com/v1/audio-analysis/7795...,0.677,239317.0,0.485,7795WJLVKJoAyVoOtCWqXN,...,0.493,306,101,82,2014-01-01,2014,7795WJLVKJoAyVoOtCWqXN,I'm The One,I'm Not The Only One,0
4,YG,1416297,83,237240,0.00552,https://api.spotify.com/v1/audio-analysis/0ZNr...,0.745,237240.0,0.346,0ZNrc4kNeQYD9koZ3KvCsy,...,0.112,551,364,85,2018-08-03,2018,0ZNrc4kNeQYD9koZ3KvCsy,Stay,"BIG BANK (feat. 2 Chainz, Big Sean, Nicki Minaj)",0


In [17]:
full_song_list.to_csv("Song_Data_ML_2000_2017.CSV",index = False)

In [18]:
#get song features for 2018 top song(for Predict)
#source:https://www.officialcharts.com/chart-news/the-official-top-40-biggest-songs-of-2018-so-far__22432/
#https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2018
df_2018_list = pd.read_csv("2018_Top_Song.csv")
df_2018_list.head()

,Title,Artist
0,Havana,Camila Cabello featuring Young Thug
1,No Limit,G-Eazy featuring ASAP Rocky and Cardi B
2,Bad at Love,Halsey
3,High Hopes,Panic! at the Disco
4,"Thank U, Next",Ariana Grande


In [19]:
#get only the list of title column
song2018_list = df_2018_list["Title"].tolist()
song2018_list

['Havana',
 'No Limit',
 'Bad at Love',
 'High Hopes',
 'Thank U, Next',
 'Without Me',
 'Mo Bamba',
 'Sunflower',
 'Zeze',
 'Mia',
 'Drip Too Hard',
 'Shallow',
 'Happier',
 'Mona Lisa',
 "Don't Cry",
 'Uproar',
 'Let It Fly',
 'Killshot',
 'Youngblood',
 'I Love It',
 'Lucky You',
 'The Ringer',
 'Love Lies',
 'God Is a Woman',
 'Sicko Mode',
 'Stargazing',
 'No Brainer',
 'Fefe',
 'Taste',
 'Nonstop',
 'In My Feelings',
 "I'm Upset",
 'Emotionless',
 "Don't Matter to Me",
 'Girls Like You',
 'Yikes',
 'Lucid Dreams',
 'Yes Indeed',
 "Boo'd Up",
 'Fake Love',
 'This Is America',
 'Never Be the Same',
 'Better Now',
 'No Tears Left to Cry',
 'ATM',
 "Kevin's Heart",
 'KOD',
 'Chun-Li',
 'Nice for What',
 'I Like It',
 'Call Out My Name',
 'Walk It Talk It',
 'Sad!',
 'Freaky Friday',
 'The Middle',
 'Psycho',
 'Look Alive',
 'All the Stars',
 'Pray for Me',
 'Stir Fry',
 'Meant to Be',
 'Say Something',
 "God's Plan",
 'Diplomatic Immunity',
 'New Rules',
 'Finesse',
 'Filthy',
 'Boda

In [20]:
#get song features by Spotify
song2018_features=[]


for song in song2018_list:
    try:
        result = sp.search(song,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":song,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "lyric_total_word_count":lyrice_total_count,
            "lyric_unique_word_count":lyrice_unique_count,
            "features":features
        }
    
        song2018_features.append(song_features)
    except:
            pass    
    
print(song2018_features)

retrying ...1secs
[{'song_name': 'Havana', 'track': 'Havana', 'artist': 'Camila Cabello', 'song_id': '1rfofaqEpACxVEHIZBJe6W', 'popularity': 87, 'duration_ms': 217306, 'release_date': '2018-01-12', 'release_year': 2018, 'artist_followers': 7359790, 'artist_popularity': 88, 'lyric_total_word_count': 626, 'lyric_unique_word_count': 180, 'features': {'danceability': 0.765, 'energy': 0.523, 'key': 2, 'loudness': -4.333, 'mode': 1, 'speechiness': 0.03, 'acousticness': 0.184, 'instrumentalness': 3.56e-05, 'liveness': 0.132, 'valence': 0.394, 'tempo': 104.988, 'type': 'audio_features', 'id': '1rfofaqEpACxVEHIZBJe6W', 'uri': 'spotify:track:1rfofaqEpACxVEHIZBJe6W', 'track_href': 'https://api.spotify.com/v1/tracks/1rfofaqEpACxVEHIZBJe6W', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1rfofaqEpACxVEHIZBJe6W', 'duration_ms': 217307, 'time_signature': 4}}, {'song_name': 'No Limit', 'track': 'No Limit', 'artist': 'G-Eazy', 'song_id': '2Xqd0wUttjueBfdcltADOv', 'popularity': 77, 'duration

In [22]:
sep = '::::' # string that doesn't appear in column names

df_song2018_features = json_normalize(song2018_features, sep=sep)
df_song2018_features.columns = df_song2018_features.columns.str.split(sep).str[-1]
df_song2018_features.head(20)

,artist,artist_followers,artist_popularity,duration_ms,acousticness,analysis_url,danceability,duration_ms,energy,id,...,uri,valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track
0,Camila Cabello,7359790,88,217306,0.1840,https://api.spotify.com/v1/audio-analysis/1rfo...,0.765,217307,0.523,1rfofaqEpACxVEHIZBJe6W,...,spotify:track:1rfofaqEpACxVEHIZBJe6W,0.394,626,180,87,2018-01-12,2018,1rfofaqEpACxVEHIZBJe6W,Havana,Havana
1,G-Eazy,3555960,86,245386,0.0117,https://api.spotify.com/v1/audio-analysis/2Xqd...,0.838,245387,0.771,2Xqd0wUttjueBfdcltADOv,...,spotify:track:2Xqd0wUttjueBfdcltADOv,0.405,990,252,77,2017-09-07,2017,2Xqd0wUttjueBfdcltADOv,No Limit,No Limit
2,Halsey,4372908,91,181279,0.0604,https://api.spotify.com/v1/audio-analysis/7y9i...,0.675,181279,0.751,7y9iMe8SOB6z3NoHE2OfXl,...,spotify:track:7y9iMe8SOB6z3NoHE2OfXl,0.612,524,150,79,2017-06-02,2017,7y9iMe8SOB6z3NoHE2OfXl,Bad at Love,Bad At Love
3,Panic! At The Disco,5397719,88,190946,0.1930,https://api.spotify.com/v1/audio-analysis/1rqq...,0.579,190947,0.904,1rqqCSm0Qe4I9rUvWncaom,...,spotify:track:1rqqCSm0Qe4I9rUvWncaom,0.681,544,124,90,2018-06-22,2018,1rqqCSm0Qe4I9rUvWncaom,High Hopes,High Hopes
4,Ariana Grande,20732435,96,207333,0.2800,https://api.spotify.com/v1/audio-analysis/2rPE...,0.724,207333,0.647,2rPE9A1vEgShuZxxzR2tZH,...,spotify:track:2rPE9A1vEgShuZxxzR2tZH,0.435,586,175,100,2018-11-03,2018,2rPE9A1vEgShuZxxzR2tZH,"Thank U, Next","thank u, next"
5,Halsey,4372908,91,201660,0.2970,https://api.spotify.com/v1/audio-analysis/5p7u...,0.752,201661,0.488,5p7ujcrUXASCNwRaWNHR1C,...,spotify:track:5p7ujcrUXASCNwRaWNHR1C,0.533,510,142,96,2018-10-04,2018,5p7ujcrUXASCNwRaWNHR1C,Without Me,Without Me
6,Sheck Wes,183535,83,183906,0.1940,https://api.spotify.com/v1/audio-analysis/1xzB...,0.729,183907,0.625,1xzBco0xcoJEDXktl7Jxrr,...,spotify:track:1xzBco0xcoJEDXktl7Jxrr,0.261,417,166,93,2018-10-05,2018,1xzBco0xcoJEDXktl7Jxrr,Mo Bamba,Mo Bamba
7,Kodak Black,3886241,91,228653,0.0710,https://api.spotify.com/v1/audio-analysis/7l3E...,0.826,225199,0.615,7l3E7lcozEodtVsSTCkcaA,...,spotify:track:7l3E7lcozEodtVsSTCkcaA,0.543,613,412,95,2018-10-12,2018,7l3E7lcozEodtVsSTCkcaA,Zeze,ZEZE (feat. Travis Scott & Offset)
8,Bad Bunny,9481969,95,210367,0.0141,https://api.spotify.com/v1/audio-analysis/116H...,0.817,210368,0.539,116H0KvKr2Zl4RPuVBruDO,...,spotify:track:116H0KvKr2Zl4RPuVBruDO,0.158,613,412,97,2018-10-11,2018,116H0KvKr2Zl4RPuVBruDO,Mia,MIA (feat. Drake)
9,Lil Baby,1000897,91,145603,0.0945,https://api.spotify.com/v1/audio-analysis/1Bxk...,0.900,145603,0.653,1BxkZE73h9BN3qwuA15TA3,...,spotify:track:1BxkZE73h9BN3qwuA15TA3,0.399,657,192,90,2018-09-12,2018,1BxkZE73h9BN3qwuA15TA3,Drip Too Hard,Drip Too Hard


In [23]:
df_song2018_features.to_csv("2018_top_song_features_1.CSV",index = False)